# Домашняя работа 6. Бустинг

 **Выполнил Дмитрий Головинов**

Максимальная оценка 10 баллов

Ответы на вопросы пишите в комментариях или в markdown ячейках. Таким же образом обозначайте блоки кода для лучшей читаемости (например, "Обучим бэггинг на логистических регрессиях : ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ").

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка: 10**

In [ ]:
# !pip install numpy
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

### Задание 1. Градиентный бустинг своими руками  (4 балла)

Вам нужно реализовать упрощенный вариант градиентного бутсинга для задачи регресси.


**Напоминание, как это работает:**

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается так:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu \in [0, 1]$ — темп обучения (гиперпараметр), $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма в ходе решения оптимизационной задачи:

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$


Заметьте, что в формуле выше нет $\nu$. Этот гиперпараметр используется для сокращения длины шага, оптимального при составлении композиции $a_N$. Идея отклонения от оптимума должна быть вам уже знакома как способ борьбы с переобучением, когда мы специально форсим модель работать чуть хуже, чем могла бы, на текущем шаге, чтобы сохранить обобщающую способность и не подогнаться под тренировочную выборку (или под шум).

С потерей в 0.5 балла можете принять $\gamma_N = 1$ для каждого $N$. На полный балл необходимо реализовать нахождение оптимального $\gamma_N$ на каждом шаге.

В качестве функции потерь $L$ возьмите MSE.

В качестве базовой модели можете использовать `DecisionTreeRegressor` из `sklearn`.
Для решения оптимизационной задачки можно воспользоваться алгоритмами из любых библиотек, например, `scipy.optimize`, или найти оптимум перебором по сетке из некоторого разумного диапазона.

Можно дописывать свои функции, если необходимо.

In [ ]:
class GradientBoosting:
    def __init__(
        self,
        base_model_class: object = DecisionTreeRegressor,
        base_model_params: dict = {'max_depth': 8},
        n_estimators: int = 500,
        learning_rate: float = 0.025
    ):
        """
        Args:
          base_model_class: Class of the base learner.

          base_model_params: Hyperparameters of the base learner.

          n_estimators: Number of boosting stages.

          learning_rate: Value used to shrink contribution of each base learner to the model.
        """
        # Инициализация основных параметров модели
        self.base_model_class = base_model_class
        self.base_model_params = base_model_params
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate

        # Инициализация массивов для хранения значений гамма и базовых моделей
        self.gamma_values = []
        self.base_models = []

    def find_optimal_gamma(self,
                           y: np.array,
                           old_predictions: np.array,
                           new_predictions: np.array) -> float:
        """
        Args:
          y: Target variable.

          old_predictions: Prediction of the additive model at the previous stage.

          new_predictions: Prediction of the base learner at the current stage.

        Returns:
          Optimal value for gamma.
        """
        # Функция потерь для оптимизации гамма
        def loss_function(gamma):
            return mean_squared_error(y, old_predictions + gamma * new_predictions)

        initial_guess = 1.0
        result = minimize(loss_function, initial_guess, method='Nelder-Mead')
        optimal_gamma = result.x[0]

        return optimal_gamma

    def _fit_base_model(self, X: np.ndarray, y: np.array):
        """
        Args:
          X: Feature matrix

          y: Target variable.

        Returns:
          Fitted base learner.
        """
        # Обучение базовой модели
        model = self.base_model_class(**self.base_model_params)
        model.fit(X, y)
        return model

    def fit(self, X: np.ndarray, y: np.array):
        """
        Args:
          X: Feature matrix

          y: Target variable.

        Returns:
          Fitted boosting.
        """
        # Процесс обучения ансамбля моделей
        predictions = np.zeros(len(y))

        for _ in range(self.n_estimators):
            # Вычисление остатков
            residuals = y - predictions
            # Обучение дополнительной модели
            model = self._fit_base_model(X, residuals)
            # Получение предсказаний от модели
            new_predictions = model.predict(X)
            # Добавление модели в список
            self.base_models.append(model)
            # Оптимизация гаммы
            gamma_opt = self.find_optimal_gamma(y, predictions, new_predictions)
            self.gamma_values.append(gamma_opt)
            # Обновление общих предсказаний
            predictions += self.learning_rate * gamma_opt * new_predictions

        return self

    def predict(self, X: np.ndarray):
        """
        Args:
          X: Feature matrix

        Returns:
          Prediction of fitted boosting.
        """
        # Получение предсказаний от ансамбля
        predictions = np.zeros(X.shape[0])

        for gamma, model in zip(self.gamma_values, self.base_models):
            predictions += self.learning_rate * gamma * model.predict(X)
        return predictions


gb = GradientBoosting(base_model_class=DecisionTreeRegressor,
                      base_model_params={'max_depth': 8},
                      n_estimators=500,
                      learning_rate=0.025) # Ручками подобрал параметры
gb.fit(X_train, y_train)
gb_mse = mean_squared_error(y_test, gb.predict(X_test))
print(f"Результат: {gb_mse}")
print(f"Выдает ±0.165, что меньше чем Random Forest (0.167) - это хорошо")

Результат: 0.3655720274211312
Выдает ±0.165, что меньше чем Random Forest (0.167) - это хорошо


Проверьте вашу реализацию на бостонском датасете. Подберите оптимальные гиперпараметры, чтобы победить RandomForestRegressor (не меняйте параметры сида).

In [ ]:
housing = fetch_california_housing()
X = housing.data[:5000]
y = housing.target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=4, n_estimators=640, random_state=19052019)

rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test))

0.1673784076585523

### Задание 2. Сравнение подходов (3 балла)

Скачайте данные о выдаче кредитов. Это данные с kaggle, целевая переменная `y` показывает, вернуло ли кредит физическое лицо.

In [ ]:
!wget  -O 'bank_data.csv' -q 'https://www.dropbox.com/s/uy27mctxo0gbuof/bank_data.csv?dl=0'

In [ ]:
df = pd.read_csv('bank_data.csv')
df.sample(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
490,32,entrepreneur,single,professional.course,no,yes,no,telephone,aug,fri,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.905,4991.6,-1
227,31,services,single,basic.6y,no,no,no,telephone,jun,wed,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,-1
3567,43,blue-collar,married,basic.6y,no,yes,no,cellular,apr,fri,...,1,999,1,failure,-1.8,93.075,-47.1,1.405,5099.1,-1
6249,26,self-employed,married,university.degree,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,-1
951,39,admin.,married,university.degree,no,no,no,cellular,jul,mon,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,-1


Решите задачу предсказания возвращения кредита методами, перечисленными ниже:

- Случайный лес
- Бэггинг на деревьях (поставьте для базовых деревьев min_samples_leaf=1)
- Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)
- Бэггинг на логистических регрессиях

Используйте логистическую регрессию, случайный лес, `GradientBoostingClassifier` и `BaggingClassifier` из `sklearn`.

1) Какая из моделей имеет лучшее качество? С чем это связано?

2) Какая из моделей сильнее всего переобучается?


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Преобразование категориальных признаков
df_dummies = pd.get_dummies(df)

# Подготовка данных
X = df_dummies.drop('y', axis=1)
y = df_dummies['y']

#Отнормируем данные
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Обучение моделей и оценка их качества
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Bagging on Trees": BaggingClassifier(estimator=DecisionTreeClassifier(min_samples_leaf=1), random_state=42),
    "Bagging with Boosting": BaggingClassifier(estimator=GradientBoostingClassifier(n_estimators=100), random_state=42),
    "Bagging on Logistic Regression": BaggingClassifier(estimator=LogisticRegression(), random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"{name}: Точность: {scores.mean()} (+/- {scores.std() * 2})")

# Качество на обучающей и тестовой выборке
for name, model in models.items():
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"{name}: На обучающей выборке: {train_accuracy}, На тестовой выборке: {test_accuracy}")


Random Forest: Точность: 0.8887005388760587 (+/- 0.007207176933175411)
Bagging on Trees: Точность: 0.8763845561674662 (+/- 0.009415787544622802)
Bagging with Boosting: Точность: 0.8891613667318055 (+/- 0.011061260887802703)
Bagging on Logistic Regression: Точность: 0.8759232545745247 (+/- 0.015695172520400473)
Random Forest: На обучающей выборке: 1.0, На тестовой выборке: 0.8811063218390804
Bagging on Trees: На обучающей выборке: 0.9921490147783252, На тестовой выборке: 0.8742816091954023
Bagging with Boosting: На обучающей выборке: 0.9005541871921182, На тестовой выборке: 0.884698275862069
Bagging on Logistic Regression: На обучающей выборке: 0.8802339901477833, На тестовой выборке: 0.8674568965517241


**ОТВЕТЫ НА ВОПРОСЫ** 1) Лучшие модели это Random forest и Bagging with Boosting. Хотя разница между результатами моделей на тестовых выборках мала <0.01, если предположить что это значимо, возможно это связано с меньшим подверженностям выбросам.

2) Я бы не сказал, что кто-то особенно переучился, ведь по определению, переобучение это хорошая работа на обучающей выборке и плохая работа на тестовой. В данном случае, все модели показали хорошую работу на тестовой выборке. Сильнее всего "переобучился" на обучающей выборке Random Forest (100%).

### Задание 3. Современные бустинги (3 балла)

Сравните на этих данных любую из трёх популярных имплементаций градиентного бустинга (xgboost, lightgbm, catboost). Подберите основные гиперпараметры (число деревьев, длина шага, глубина дерева/число листьев). Получилось ли круче, чем с моделями выше?

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier

# Преобразование категориальных переменных
df_dummies = pd.get_dummies(df)
X = df_dummies.drop('y', axis=1)
y = df_dummies['y']

# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


# Создание и обучение модели LightGBM
lgbm = LGBMClassifier(**params)
lgbm.fit(X_train, y_train)

# Оценка точности модели
lgbm_accuracy = cross_val_score(lgbm, X_train, y_train, cv=5).mean()
print(f"Optimized LightGBM Accuracy: {lgbm_accuracy}")
print("Выдает >0.89, что лучше, чем предыдущие результаты")

[LightGBM] [Info] Number of positive: 3269, number of negative: 3227
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003723 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 777
[LightGBM] [Info] Number of data points in the train set: 6496, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503233 -> initscore=0.012931
[LightGBM] [Info] Start training from score 0.012931
[LightGBM] [Info] Number of positive: 2615, number of negative: 2581
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 769
[LightGBM] [Info] Number of data points in the train set: 5196, number of used features: 60
[LightGBM] [Info] [binary:Bo